In [1]:
from theano import clone, shared
import theano.tensor as T
from theano import gof
import theano
import numpy as np

In [2]:
a = shared(np.array([4,3]), "a")
b = shared(np.array([1,3]), "b")

In [3]:
b.type

TensorType(int64, vector)

In [4]:
ab = a + b
ab.name="ab"

In [5]:
aab = (a + ab) + ab

In [6]:
b2 = shared(np.array([1,3]), "b2")
b3 = shared(np.array([1,3]), "b3")
b4 = shared(np.array([1,3]), "b4")

In [7]:
b2.type

TensorType(int64, vector)

# clone

In [ ]:
aab2 = clone(aab, replace={b:b2})

In [ ]:
aab2 == aab

In [ ]:
variables = gof.graph.variables(gof.graph.inputs([aab2]), [aab2])
variables

In [ ]:
[v is ab for v in variables]

In [ ]:
gof.graph.variables()

# myproxy

In [8]:
from schlichtanders.mymeta import proxify

In [9]:
proxify(b, b2)

b2

In [25]:
b.__original__

b

In [ ]:
%debug

In [10]:
proxify(b2, b3)

b3

In [11]:
b3 == b2

True

In [12]:
b2 == b3

True

In [ ]:
proxify(b, b2)

In [ ]:
b == b2

In [ ]:
proxify(b2, b3)

In [ ]:
b2

In [ ]:
b == b3

In [13]:
b.hallo = "hallo"

In [14]:
b3.hallo

'hallo'

In [19]:
theano.pp(aab)

'((a + (a + b4)) + (a + b4))'

In [ ]:
hasattr(b.__subject__.__subject__, "__subject__")

In [18]:
proxify(b, b)

b4

In [22]:
type(b3.__original__)

theano.tensor.sharedvar.TensorSharedVariable

In [16]:
proxify(b, b4)

b4

In [ ]:
type(b3.__subject__)

In [ ]:
type(get_subject(b))

# morph

In [ ]:
from wrapt import ObjectProxy

In [ ]:
from schlichtanders.mymeta import morph

In [ ]:
def morph_proxy(a, b):
    return morph(a, ObjectProxy(b))

In [ ]:
pb2 = ObjectProxy(b2)

In [ ]:
dir(pb2)

In [ ]:
morph_proxy(b, b2)

In [ ]:
morph(b, b2)

In [ ]:
morph_proxy(b2, b3)

In [ ]:
theano.pp(aab)

In [ ]:
b == b2

In [ ]:
hash??

In [ ]:
b.__hash__(), hash(b)

In [ ]:
id(b)

In [ ]:
hash(b)

In [ ]:
hash(b2)

# adapt hash

In [ ]:
class Proxy(object):
    
    """Delegates all operations (except ``.__subject__``) to another object"""
    
    def __init__(self,subject):
        self.__subject__ = subject


    def __call__(self,*args,**kw):
        return self.__subject__(*args,**kw)

    def __getattribute__(self, attr, oga=object.__getattribute__):
        subject = oga(self,'__subject__')
        if attr=='__subject__':
            return subject
        return getattr(subject,attr)

    def __setattr__(self,attr,val, osa=object.__setattr__):
        if attr=='__subject__':
            osa(self,attr,val)
        else:
            setattr(self.__subject__,attr,val)

    def __delattr__(self,attr, oda=object.__delattr__):
        if attr=='__subject__':
            oda(self,attr)
        else:
            delattr(self.__subject__,attr)

    def __nonzero__(self):
        return bool(self.__subject__)

    def __getitem__(self,arg):
        return self.__subject__[arg]

    def __setitem__(self,arg,val):
        self.__subject__[arg] = val

    def __delitem__(self,arg):
        del self.__subject__[arg]

    def __getslice__(self,i,j):
        return self.__subject__[i:j]


    def __setslice__(self,i,j,val):
        self.__subject__[i:j] = val

    def __delslice__(self,i,j):
        del self.__subject__[i:j]

    def __contains__(self,ob):
        return ob in self.__subject__

    for name in 'repr str hash len abs complex int long float iter oct hex'.split():
        exec "def __%s__(self): return %s(self.__subject__)" % (name,name)

    for name in 'cmp', 'coerce', 'divmod':
        exec "def __%s__(self,ob): return %s(self.__subject__,ob)" % (name,name)

    for name,op in [
        ('lt','<'), ('gt','>'), ('le','<='), ('ge','>='),
        ('eq','=='), ('ne','!=')
    ]:
        exec "def __%s__(self,ob): return self.__subject__ %s ob" % (name,op)

    for name,op in [('neg','-'), ('pos','+'), ('invert','~')]:
        exec "def __%s__(self): return %s self.__subject__" % (name,op)

    for name, op in [
        ('or','|'),  ('and','&'), ('xor','^'), ('lshift','<<'), ('rshift','>>'),
        ('add','+'), ('sub','-'), ('mul','*'), ('div','/'), ('mod','%'),
        ('truediv','/'), ('floordiv','//')
    ]:
        exec (
            "def __%(name)s__(self,ob):\n"
            "    return self.__subject__ %(op)s ob\n"
            "\n"
            "def __r%(name)s__(self,ob):\n"
            "    return ob %(op)s self.__subject__\n"
            "\n"
            "def __i%(name)s__(self,ob):\n"
            "    self.__subject__ %(op)s=ob\n"
            "    return self\n"
        )  % locals()

    del name, op

    # Oddball signatures

    def __rdivmod__(self,ob):
        return divmod(ob, self.__subject__)

    def __pow__(self,*args):
        return pow(self.__subject__,*args)

    def __ipow__(self,ob):
        self.__subject__ **= ob
        return self

    def __rpow__(self,ob):
        return pow(ob, self.__subject__)

In [ ]:
class MyObjectProxy(AbstractProxy):
    """Proxy for a specific object"""



In [ ]:
class A(object):
    pass

In [ ]:
a = A()
b = A()

In [ ]:
a.__class__ = MyObjectProxy
MyObjectProxy.__init__(a, b)

In [ ]:
a == b

In [ ]:
a == b

In [ ]:
hash(a), hash(b)

In [ ]:
class X(object):
    def __test__(self):
        print "class method"

x = X()
x.__test__ = lambda : "instance function"
y = X()
x.__test__()